<a href="https://colab.research.google.com/github/Samoed/CourseWork2/blob/main/TestApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U bert-extractive-summarizer
!pip install -U transformers

     |████████████████████████████████| 2.0MB 5.8MB/s 
     |████████████████████████████████| 3.2MB 20.1MB/s 
     |████████████████████████████████| 890kB 37.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=49065c64982f326a5408e879cbadf0efef84c38382b1a2494cc0e08d1449245c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Requirement already up-to-date: transformers in /usr/local/lib/python3.7/dist-packages (4.4.2)


In [24]:
!unzip embeddingsShort.zip

Archive:  embeddingsShort.zip
  inflating: embeddingsPickle/ Абашева Александра Сергеевна.data  
  inflating: embeddingsPickle/ Андреева Ольга Юрьевна.data  
  inflating: embeddingsPickle/ Артемьев Дмитрий Геннадьевич.data  
  inflating: embeddingsPickle/ Барахас Анхель.data  
  inflating: embeddingsPickle/ Белых Светлана Александровна.data  
  inflating: embeddingsPickle/ Божья-Воля Анастасия Александровна.data  
  inflating: embeddingsPickle/ Бузмаков Алексей Владимирович.data  
  inflating: embeddingsPickle/ Букина Татьяна Витальевна.data  
  inflating: embeddingsPickle/ Быкова Анна Андреевна.data  
  inflating: embeddingsPickle/ Василенко Юрий Владимирович.data  
  inflating: embeddingsPickle/ Викентьева Ольга Леонидовна.data  
  inflating: embeddingsPickle/ Володина Галина Емельяновна.data  
  inflating: embeddingsPickle/ Гагарина Динара Амировна.data  
  inflating: embeddingsPickle/ Гергерт Дмитрий Владимирович.data  
  inflating: embeddingsPickle/ Гущин Валерий Рафаилович.data  

In [10]:
import requests
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig
from tqdm import tqdm
from summarizer import Summarizer

import pickle
import re
import math
import os

In [11]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

custom_config = BertConfig.from_pretrained('DeepPavlov/rubert-base-cased')
custom_config.output_hidden_states=True
custom_tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
custom_model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', config=custom_config)
#custom_model.to('cuda')
custom_model.eval()
#torch.cuda.get_device_name(0)
modelSum = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [39]:
path = '/content/embeddingsPickle'
directories = os.listdir(path)
emb = {}
for file in directories:
   with open(path+'/'+file, "rb") as f:
    arr = pickle.load(f)
    emb[file.split('.')[0]] = arr

61

In [20]:
with open("/content/TestDoc", 'r') as f:
    text = f.read()
    text = "".join([i.replace('\n',' ').replace('\t',' ') for i in text])
text
embeddings = modelSum.run_embeddings(text, num_sentences=10)

In [51]:
error = {}
for name, arr in tqdm(emb.items()):
    sum = 0
    avg = 0
    mae = 0
    rmae = 0
    mse = 0
    rmse = 0
    for i in range(11):
        for arrRow in range(len(arr)):
            for j in range(768):
                sum += embeddings[i][j] - arr[arrRow][j]
                mae += abs(embeddings[i][j] - arr[arrRow][j])
                rmae += abs(embeddings[i][j] - arr[arrRow][j])**0.5
                mse += (embeddings[i][j] - arr[arrRow][j])**2
                rmse += ((embeddings[i][j] - arr[arrRow][j])**2)**0.5
    error[mae] = name

100%|██████████| 61/61 [01:27<00:00,  1.44s/it]


In [53]:
error[min(sums)]

' Кушев Вадим Олегович'

In [58]:
error[min(error.keys())]

' Кушев Вадим Олегович'